In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold,GroupKFold
import math
import scipy
import numpy as np
import pandas as pd
import sklearn.metrics as mtr 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout, PReLU, BatchNormalization, ELU, GaussianNoise, Activation
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
from sklearn import preprocessing
import gc
import os
from tqdm import tqdm_notebook
import keras.backend as K
import lightgbm as lgb
#note： 
#1. As a result it might not be worthwhile to use features related to game clock/quarter of the game。
#2. There is no relationships between number of rushes before and running yards gained。
#3. rushing success larger depends on defender in box, or defender that are close to offensive lineman and attempt 
#   to counter the blocking.
#4. highly drafted player has the same average rushing yards as the rest

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv(r'C:\Users\38980\OneDrive\Desktop\study\kaggle\NFL\train.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#team abbreviations correct
def data_clean(df):
#correct name   
    df.loc[df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
    df.loc[df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
    df.loc[df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
    df.loc[df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
    df.loc[df['FieldPosition'] == 'ARZ', 'FieldPosition'] = 'ARI'
    df.loc[df['FieldPosition'] == 'BLT', 'FieldPosition'] = 'BAL'
    df.loc[df['FieldPosition'] == 'CLV', 'FieldPosition'] = 'CLE'
    df.loc[df['FieldPosition'] == 'HST', 'FieldPosition'] = 'HOU'

    df.loc[df['Season'] == 2017, 'S'] = (df.loc[df['Season'] == 2017, 'S'] - 2.4355) / 1.2930 * 1.4551 + 2.7570


# offense time and defence time
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    
#time
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df['GameClock'] = df['GameClock'].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
#player height
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

#weather
    def map_weather(txt):
        ans = 1
        if pd.isna(txt):
            return 0
        if 'partly' in txt:
            ans*=0.5
        if 'climate controlled' in txt or 'indoor' in txt:
            return ans*3
        if 'sunny' in txt or 'sun' in txt:
            return ans*2
        if 'clear' in txt:
            return ans
        if 'cloudy' in txt:
            return -ans
        if 'rain' in txt or 'rainy' in txt:
            return -2*ans
        if 'snow' in txt:
            return -3*ans
        return 0
    df['Cleaned_GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['Cleaned_GameWeather'].apply(map_weather)

#diff Score    
    df["DiffScoreBeforePlay_ob"] = (df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"])
    df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]] = - df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]]

#left to right
    df['New_X'] = df['X']
    df.loc[df['PlayDirection'] == 'left','New_X'] = 120 - df.loc[df['PlayDirection'] == 'left','X']
    df['New_Y'] = df['Y']
    df.loc[df['PlayDirection'] == 'left','New_Y'] = 160/3 - df.loc[df['PlayDirection'] == 'left','Y']
    df['Orientation_std'] = df['Orientation']
    #df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std'] = 360 - df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std']
    df.loc[df['PlayDirection'] == 'left', 'Orientation_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation_std'], 360)
    df['Dir_std'] = df['Dir']
    #df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = 360 - df.loc[df['PlayDirection'] == 'left', 'Dir_std']
    df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir_std'], 360)
    df['YardLine_std'] = 100 - df['YardLine']
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine_std'
         ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine']
    df["Orientation_sin"] = df["Orientation_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Orientation_cos"] = df["Orientation_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    df["Dir_sin"] = df["Dir_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Dir_cos"] = df["Dir_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

#distance and S
    
    #distance to yardline
    df['Dis_YardLine'] = df['New_X'] - df['YardLine_std'] - 10
    
    #distance to rusher
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    Rusher =df.loc[df['IsRusher'],['PlayId','New_X','New_Y','Dir_std','S']].rename(columns={"New_X":"Rusher_X","New_Y":"Rusher_Y",'PossessionTeam':'Offense_Team','Dir_std':'Rusher_Dir_std','S':'Rusher_Speed'})
    df = df.merge(Rusher,how = 'left',on = 'PlayId')
    df['Distance_to_Rusher'] = df[["New_X","Rusher_X","New_Y","Rusher_Y"]].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_to_Rusher'] = df[["New_X","Rusher_X","New_Y","Rusher_Y"]].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_Diff'] = df['Degree_to_Rusher'] - df['Rusher_Dir_std']
    df['Degree_Diff2'] = 270 - (90-df['Degree_to_Rusher']) - df['Dir_std']
    df['Speed_Ratio'] =  (df['Rusher_Speed']+0.01)/(df['S']+0.01)

# speed
    df['Horizontal Speed'] = df['S']*df["Dir_sin"]
    df['Vertical Speed'] = df['S']*df["Dir_cos"]



    
    return df

In [4]:
train = data_clean(train_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in remainder
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in remainder
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


In [5]:
temp = train.drop(train[(train['Position'].isin(['FS','S','SS'])) & (train['Distance_to_Rusher']>15)].index)
temp = temp.merge(train.loc[train['IsRusher'],['PlayId','Dir_std']]\
                  .rename(columns={'Dir_std':'Rusher_Dir'}), on = 'PlayId',how = 'left')

#temp2 = temp.loc[(temp['IsOnOffense'])]
temp = temp.loc[(~temp['IsOnOffense'])]


op_up = temp[((temp['Rusher_Dir']<90)|(temp['Rusher_Dir']>270))&(temp['New_Y']>=temp['Rusher_Y'])]
op_up = op_up.sort_values(['PlayId','New_Y'])
op_up[['Next_Y','Next_X']] = op_up[['New_Y','New_X']].shift(1,axis = 0)
op_up['Gap'] = op_up['New_Y'] - op_up['Next_Y']
op_up[['PlayId','New_Y','Next_Y','Gap']]
op_up = op_up[op_up['Gap'] > 0]
closest_op_up = op_up[['PlayId','Gap']].groupby('PlayId').head(1).reset_index()
best_up = op_up[['PlayId','Gap']].groupby('PlayId').max().reset_index()
def_num_up = op_up[['PlayId','Gap']].rename(columns={'Gap': 'num_def'}).groupby('PlayId').count().reset_index()

op_down = temp[(~((temp['Rusher_Dir']<90)|(temp['Rusher_Dir']>270)))&(temp['New_Y']<temp['Rusher_Y'])]
op_down = op_down.sort_values(['PlayId','New_Y'])
op_down[['Next_Y','Next_X']] = op_down[['New_Y','New_X']].shift(1,axis = 0)
op_down['Gap'] = op_down['New_Y'] - op_down['Next_Y']
op_down[['PlayId','New_Y','Next_Y','Gap']]
closest_op_down = op_down[['PlayId','Gap']].groupby('PlayId').tail(1).reset_index()
best_down = op_down[['PlayId','Gap']].groupby('PlayId').max().reset_index()
def_num_down = op_down[['PlayId','Gap']].rename(columns={'Gap': 'num_def'}).groupby('PlayId').count().reset_index()



closest_op = closest_op_up.append(closest_op_down)
best = best_up.append(best_down)
closest_op[['PlayId','Gap']]

,PlayId,Gap
0,20170907000118,1.200000
1,20170907000189,0.460000
2,20170907000516,0.600000
3,20170907000917,0.630000
4,20170907001077,1.460000
5,20170907001156,0.930000
6,20170907001488,0.410000
7,20170907001509,2.250000
8,20170907001530,0.910000
9,20170907001605,0.130000


In [6]:
closest_op[closest_op['Gap'] < 0]

,index,PlayId,Gap
1765,74330,20171012000641,-17.266667
2514,104774,20171029001345,-17.480000
2764,115313,20171030000776,-11.323333
3612,150422,20171119071455,-7.046667
4068,169975,20171127001488,-14.090000
4301,179772,20171203081200,-15.310000
5352,222285,20171224031478,-14.770000
5926,246186,20171231124047,-13.766667
6245,259490,20180909092856,-10.850000
6476,269096,20180916043849,-17.550000


In [7]:
def split_data(df):
    df1 = df.loc[df['IsRusher']]
    df2 = df.loc[df['IsOnOffense'] & (~df['IsRusher'])]
    df3 = df.loc[~df['IsOnOffense']]

# max yards
    df1['Max_Yards'] = df1['YardLine']
    df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards']
    df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards']

# min_time_to_tacke
    df3['Min_Time_Tacke'] = (df3['Distance_to_Rusher']+0.01)/(df3['S']+0.01)
    df3.loc[df3['Min_Time_Tacke'] == np.inf, 'Min_Time_Tacke'] = 20

# defence_X_Y_spread
    Defence_X_Y_std = df3[["PlayId",'New_X','New_Y']].groupby("PlayId").std().rename(columns={'New_X':'Defense_X_std','New_Y':'Defense_Y_std'}) \
    .reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed2_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed2_std'}).reset_index()

    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed2_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed2_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed4_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed4_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed4_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Defence_X_Y_std, how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed4_std,how = 'left',  on ='PlayId')

#distance to QB
    dis_QB = df2.loc[df2["Position"] =='QB',['PlayId','Distance_to_Rusher']].groupby(['PlayId']).mean().rename(columns={'Distance_to_Rusher':'dis_to_QB'})
    df1 = df1.merge(dis_QB,how = 'left', on='PlayId')

#defence min,max,mean,std distance to rusher
    stat = df3.groupby(['GameId','PlayId']).agg({'Distance_to_Rusher':['min','max','mean','std']})
    stat.columns = stat.columns.droplevel()
    df1 = df1.merge(stat,how = 'left', on='PlayId')

# offense_X_Y_spread
    df2 = df2.sort_values(['PlayId','New_X'])
    Offense_X_Removed2_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed2_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed2_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed2_std'}).reset_index()
    
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_X_Removed4_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed4_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed4_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Offense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed4_std,how = 'left',  on ='PlayId')
    
# nearest offenders to defender
    dis_to_closest_offender = pd.DataFrame()
    dis_to_closest_defender = pd.DataFrame()
    for playid in df3['PlayId'].unique():
        offense = df2.loc[df2['PlayId'] == playid]
        defence = df3.loc[df3['PlayId'] == playid]
        ary = scipy.spatial.distance.cdist(defence[['New_X','New_Y']], offense[['New_X','New_Y']], metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, defence[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_offender = dis_to_closest_offender.append(ary)
        
        ary = scipy.spatial.distance.cdist(offense[['New_X','New_Y']], defence[['New_X','New_Y']],  metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, offense[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_defender = dis_to_closest_defender.append(ary)
    df3 = df3.merge(dis_to_closest_offender,how = 'left',on = ['PlayId','NflId'])
    df2 = df2.merge(dis_to_closest_defender,how = 'left',on = ['PlayId','NflId'])

#degree to closest offender
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df4 = df3.merge(df2, how = 'left', on = ['PlayId',0],suffixes=('_defender', '_offender'))
    df4['degree_to_closest_offender'] = df4[['New_X_offender','New_X_defender','New_Y_offender','New_Y_defender']].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df3['degree_to_closest_offender'] = df4['degree_to_closest_offender']

# personnel_features
    def defense_formation(l):
        dl = 0
        lb = 0
        db = 0
        other = 0

        for position in l:
            sub_string = position.split(' ')
            if sub_string[1] == 'DL':
                dl += int(sub_string[0])
            elif sub_string[1] in ['LB','OL']:
                lb += int(sub_string[0])
            else:
                db += int(sub_string[0])

        counts = (dl,lb,db,other)

        return counts
    def offense_formation(l):
        qb = 0
        rb = 0
        wr = 0
        te = 0
        ol = 0

        sub_total = 0
        qb_listed = False
        for position in l:
            sub_string = position.split(' ')
            pos = sub_string[1]
            cnt = int(sub_string[0])

            if pos == 'QB':
                qb += cnt
                sub_total += cnt
                qb_listed = True
            # Assuming LB is a line backer lined up as full back
            elif pos in ['RB','LB']:
                rb += cnt
                sub_total += cnt
            # Assuming DB is a defensive back and lined up as WR
            elif pos in ['WR','DB']:
                wr += cnt
                sub_total += cnt
            elif pos == 'TE':
                te += cnt
                sub_total += cnt
            # Assuming DL is a defensive lineman lined up as an additional line man
            else:
                ol += cnt
                sub_total += cnt

        # If not all 11 players were noted at given positions we need to make some assumptions
        # I will assume if a QB is not listed then there was 1 QB on the play
        # If a QB is listed then I'm going to assume the rest of the positions are at OL
        # This might be flawed but it looks like RB, TE and WR are always listed in the personnel
        if sub_total < 11:
            diff = 11 - sub_total
            if not qb_listed:
                qb += 1
                diff -= 1
            ol += diff

        counts = (qb,rb,wr,te,ol)

        return counts
    def split_personnel(s):
        splits = s.split(',')
        for i in range(len(splits)):
            splits[i] = splits[i].strip()

        return splits    
    def personnel_features(df):
        personnel = df[['GameId','PlayId','OffensePersonnel','DefensePersonnel']].drop_duplicates()
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: defense_formation(x))
        personnel['num_DL'] = personnel['DefensePersonnel'].apply(lambda x: x[0])
        personnel['num_LB'] = personnel['DefensePersonnel'].apply(lambda x: x[1])
        personnel['num_DB'] = personnel['DefensePersonnel'].apply(lambda x: x[2])

        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: offense_formation(x))
        personnel['num_QB'] = personnel['OffensePersonnel'].apply(lambda x: x[0])
        personnel['num_RB'] = personnel['OffensePersonnel'].apply(lambda x: x[1])
        personnel['num_WR'] = personnel['OffensePersonnel'].apply(lambda x: x[2])
        personnel['num_TE'] = personnel['OffensePersonnel'].apply(lambda x: x[3])
        personnel['num_OL'] = personnel['OffensePersonnel'].apply(lambda x: x[4])

        # Let's create some features to specify if the OL is covered
        personnel['OL_diff'] = personnel['num_OL'] - personnel['num_DL']
        personnel['OL_TE_diff'] = (personnel['num_OL'] + personnel['num_TE']) - personnel['num_DL']
        # Let's create a feature to specify if the defense is preventing the run
        # Let's just assume 7 or more DL and LB is run prevention
        personnel['run_def'] = (personnel['num_DL'] + personnel['num_LB'] > 6).astype(int)

        personnel.drop(['OffensePersonnel','DefensePersonnel'], axis=1, inplace=True)
        
        return personnel
    
    personnel = personnel_features(df1)   
    df1 = df1.merge(personnel,how = 'left',  on ='PlayId')
    
# Gap
    temp = df3.drop(df3[(df3['Position'].isin(['FS','S','SS'])) & (df3['Distance_to_Rusher']>15)].index)
    temp = df3.merge(df.loc[df['IsRusher'],['PlayId','Dir_std']]\
                      .rename(columns={'Dir_std':'Rusher_Dir'}), on = 'PlayId',how = 'left')


    op_up = temp[((temp['Rusher_Dir']<90)|(temp['Rusher_Dir']>270))&(temp['New_Y']>=temp['Rusher_Y'])]
    op_up = op_up.sort_values(['PlayId','New_Y'])
    op_up[['Next_Y','Next_X']] = op_up[['New_Y','New_X']].shift(1,axis = 0)
    op_up['Gap'] = op_up['New_Y'] - op_up['Next_Y']
    op_up = op_up[op_up['Gap'] > 0]
    closest_op_up = op_up[['PlayId','Gap']].groupby('PlayId').head(1).reset_index()
    best_up = op_up[['PlayId','Gap']].groupby('PlayId').max().reset_index()
    def_num_up = op_up[['PlayId','Gap']].rename(columns={'Gap': 'num_def'}).groupby('PlayId').count().reset_index()

    op_down = temp[(~((temp['Rusher_Dir']<90)|(temp['Rusher_Dir']>270)))&(temp['New_Y']<temp['Rusher_Y'])]
    op_down = op_down.sort_values(['PlayId','New_Y'])
    op_down[['Next_Y','Next_X']] = op_down[['New_Y','New_X']].shift(1,axis = 0)
    op_down['Gap'] = op_down['New_Y'] - op_down['Next_Y']
    closest_op_down = op_down[['PlayId','Gap']].groupby('PlayId').tail(1).reset_index()
    best_down = op_down[['PlayId','Gap']].groupby('PlayId').max().reset_index()
    def_num_down = op_down[['PlayId','Gap']].rename(columns={'Gap': 'num_def'}).groupby('PlayId').count().reset_index()



    closest_op = closest_op_up.append(closest_op_down).rename(columns = {'Gap':'Closet_Gap'})
    best = best_up.append(best_down).rename(columns = {'Gap':'Best_Gap'})
    df1 = df1.merge(closest_op, on='PlayId',how = 'left')
    df1 = df1.merge(best, on='PlayId',how = 'left')
#select useful columns    
    rusher = df1[['PlayId','TimeDelta','Team','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y', \
                 'Orientation_std','Dir_std','Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position',\
                 'Quarter','GameClock','Down','Distance','OffenseFormation',\
                  'DefendersInTheBox','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                 'Offense_X_Removed2_std','Offense_Y_Removed2_std','Offense_X_Removed4_std','Offense_Y_Removed4_std',\
                 'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std','Defense_Y_Removed2_std','Defense_X_Removed4_std',\
                 'Defense_Y_Removed4_std',\
                  'num_DL','num_LB','num_DB','num_QB','num_RB','num_WR','num_TE','num_OL','OL_diff','OL_TE_diff','run_def',\
                 'min','max','std','mean','dis_to_QB','Max_Yards','DiffScoreBeforePlay_ob','Closet_Gap','Best_Gap']]
    rusher = rusher.sort_values('PlayId')
    game = df1[['PlayId','Cleaned_GameWeather']]
    game = game.sort_values('PlayId')
    offender = df2[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff']]
    offender = offender.sort_values(['PlayId','Distance_to_Rusher'])
    defender = df3[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff','Min_Time_Tacke','Speed_Ratio','degree_to_closest_offender']+list(range(2))] #list(range(2)) two closest offenders
    defender = defender.sort_values(['PlayId','Distance_to_Rusher'])

    
    return rusher, game, offender, defender

In [8]:
rusher, game, offender, defender = split_data(train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

In [9]:
#remove some columns
rusher = rusher.drop('max',axis = 1)

offender = offender.sort_values(['PlayId','Distance_to_Rusher']).groupby('PlayId').head(2)

defender = defender.sort_values(['PlayId','Distance_to_Rusher']).drop(['New_Y','Position'],axis = 1).groupby('PlayId').head(6)

In [14]:
#encoding fit
le = preprocessing.LabelEncoder()
categories =[]
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = rusher[i].astype(str)
    categories.append(rusher[i].unique())

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = game[i].astype(str)
    categories.append(game[i].unique())

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = offender[i].astype(str)
    categories.append(offender[i].unique())

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = defender[i].astype(str)
    categories.append(defender[i].unique())
categories = np.hstack(categories)
le.fit(categories)
le.classes_

array(['ACE', 'C', 'CB', 'DE', 'DT', 'EMPTY', 'FB', 'G', 'HB', 'I_FORM',
       'JUMBO', 'NT', 'OG', 'OLB', 'OT', 'PISTOL', 'QB', 'RB', 'SHOTGUN',
       'SINGLEBACK', 'T', 'TE', 'WILDCAT', 'WR', 'away', 'home', 'nan'],
      dtype=object)

In [15]:
#encoding transform
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = le.transform(rusher[i])

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = le.transform(game[i])

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = le.transform(offender[i])

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = le.transform(defender[i])

In [16]:
#reshape offender
offender_players = [offender.drop('PlayId',axis = 1).iloc[np.arange(k, len(offender), 2)].reset_index(drop = True) for k in range(2)]
offender_players = np.hstack([t.values for t in offender_players])
offender_players.shape

(23171, 32)

In [17]:
#reshape defender
defender_players = [defender.drop('PlayId',axis = 1).iloc[np.arange(k, len(defender), 6)].reset_index(drop = True) for k in range(6)]
defender_players = np.hstack([t.values for t in defender_players])
defender_players.shape


(23171, 114)

In [18]:
rusher.shape

(23171, 54)

In [19]:
#train
train_y =train.loc[train['IsRusher'],['Yards','PlayId']].sort_values('PlayId').drop('PlayId',axis = 1)
train_x = np.hstack([rusher.drop('PlayId',axis = 1).values,defender_players,offender_players])
train_y99 =(train_y + 99).reset_index(drop=True).Yards

In [20]:
#loss function
def crps_eval(y_pred, dataset, is_higher_better=False):
    labels = dataset.get_label()
    labels = labels.astype('int')
    y_true = np.zeros((len(labels),199))
    for i, v in enumerate(labels):
        y_true[i, v:] = 1
    y_pred = y_pred.reshape(-1, 199, order='F')
    y_pred = np.clip(y_pred.cumsum(axis=1), 0, 1)
    return 'crps', np.mean((y_pred - y_true)**2), False

In [21]:
# hyper parameters
params = {'max_depth':5, 'lambda_l1': 1.4, 'lambda_l2': 1.2,
 'num_leaves': 32, 'feature_fraction': 0.4,
 'subsample': 0.4, 'min_child_samples': 15,
 'learning_rate': 0.02,
 'num_iterations': 10000, 'random_state': 42,
 'objective': 'multiclass',
 'min_gain_to_split':0.9,
 'num_class':199,
 'metric':'None'}


In [22]:
import lightgbm as lgb
for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              valid_sets=[dtrain,dvalid],
                              early_stopping_rounds=10,
                              verbose_eval=100,
                              feval=crps_eval)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds


KeyboardInterrupt: 

In [78]:
#CV score
# for leave out one feature importance compare
print(np.mean([0.0129045,0.0128677,0.0127771,0.0127333,0.013174]))

# 5 closest defense palyer drop 'New_Y','back_oriented_down_field',max
print(np.mean([0.0128208, 0.0127364, 0.0126515, 0.0126222,0.0130614]))

# all defense palyer
print(np.mean([0.012802,0.0127486,0.0126934,0.0126226,0.0130668]))

# 5 closest defender + 2 closest offender l1,l2 = 1.2
print(np.mean([0.0127868,0.0127041,0.0126144,0.0125952,0.0130483]))

# 6 closest defender + 2 closest offender with new feature
print(np.mean([0.0127472, 0.0126321, 0.012575, 0.0125473, 0.0129863]))


0.012891320000000001
0.01277846
0.01278668
0.012749759999999999
0.012697579999999997


In [10]:
#rusher = pd.DataFrame(rusher)
X_Dense = rusher[['Dis_YardLine','PlayerAge','PlayerHeight','PlayerWeight', \
            'Orientation_std','Dir_std','New_X','New_Y','Horizontal Speed',\
            'Vertical Speed','S','A','Dis','GameClock',\
            'Distance','Max_Yards',\
            'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std',\
            'Offense_Y_Removed2_std',\
            'min','std','mean','Max_Yards','dis_to_QB',\
            'HomeScoreBeforePlay','VisitorScoreBeforePlay',\
            "DiffScoreBeforePlay_ob",'Quarter','Down','Closet_Gap','Best_Gap']]
X_Dense = X_Dense.fillna(0)
yards = train.loc[train['IsRusher'],['Yards','PlayId']].\
sort_values('PlayId').drop('PlayId',axis = 1).Yards

y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1

In [11]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
from keras.losses import binary_crossentropy
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import codecs

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import f1_score




class CRPSCallback(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False):
        super(CRPSCallback, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        
        print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_score_val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_score_val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_score_val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_score_val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            val_s = np.round(val_s, 6)
            logs['CRPS_score_val'] = val_s

In [12]:
def get_model(x_tr,y_tr,x_val,y_val):
    inp = Input(shape = (x_tr.shape[1],))
    x = Dense(1024, input_dim=X.shape[1], activation='relu')(inp)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    
    out = Dense(199, activation='softmax')(x)
    model = Model(inp,out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[])
    #add lookahead
#     lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
#     lookahead.inject(model) # add into model

    
    es = EarlyStopping(monitor='CRPS_score_val', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=10)

    mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',
                                   save_best_only=True, verbose=1, save_weights_only=True)
    
    bsz = 1024
    steps = x_tr.shape[0]/bsz
    


    model.fit(x_tr, y_tr,callbacks=[CRPSCallback(validation = (x_val,y_val)),es,mc], epochs=100, batch_size=bsz,verbose=1)
    model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * x_val.shape[0])
    crps = np.round(val_s, 6)

    return model,crps

In [13]:
d = defender[['PlayId','S','A','Horizontal Speed','Vertical Speed']].fillna(0)
d = [d.drop('PlayId',axis = 1).iloc[np.arange(k, len(d), 6)].reset_index(drop = True) for k in range(6)]
d = np.hstack([t.values for t in d])



X = np.hstack([X_Dense,d])
scaler = StandardScaler()
X = scaler.fit_transform(X)
from sklearn.model_selection import train_test_split, KFold
import time

losses = []
models = []
crps_csv = []

s_time = time.time()


for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        print("-----------")
        print("-----------")
        tr_x,tr_y = X[tr_inds],y[tr_inds]
        val_x,val_y = X[val_inds],y[val_inds]
        model,crps = get_model(tr_x,tr_y,val_x,val_y)
        models.append(model)
        print("the %d fold crps is %f"%((k_fold+1),crps))
        crps_csv.append(crps)
 
print("mean crps is %f"%np.mean(crps_csv))


def predict(x_te):
    model_num = len(models)
    for k,m in enumerate(models):
        if k==0:
            y_pred = m.predict(x_te,batch_size=1024)
        else:
            y_pred+=m.predict(x_te,batch_size=1024)
            
    y_pred = y_pred / model_num
    
    return y_pred

-----------
-----------
validation shape 2
Epoch 1/100
18536/18536 [==============================] - ETA: 11s - loss: 5.87 - ETA: 6s - loss: 5.8481 - ETA: 4s - loss: 5.831 - ETA: 3s - loss: 5.810 - ETA: 2s - loss: 5.795 - ETA: 2s - loss: 5.770 - ETA: 1s - loss: 5.757 - ETA: 1s - loss: 5.746 - ETA: 1s - loss: 5.733 - ETA: 1s - loss: 5.715 - ETA: 1s - loss: 5.709 - ETA: 0s - loss: 5.694 - ETA: 0s - loss: 5.680 - ETA: 0s - loss: 5.661 - ETA: 0s - loss: 5.648 - ETA: 0s - loss: 5.634 - ETA: 0s - loss: 5.617 - ETA: 0s - loss: 5.601 - 2s 120us/step - loss: 5.6005

Epoch 00001: CRPS_score_val improved from inf to 0.08286, saving model to best_model.h5
Epoch 2/100
18536/18536 [==============================] - ETA: 1s - loss: 5.322 - ETA: 1s - loss: 5.253 - ETA: 1s - loss: 5.226 - ETA: 1s - loss: 5.201 - ETA: 1s - loss: 5.202 - ETA: 1s - loss: 5.199 - ETA: 0s - loss: 5.182 - ETA: 0s - loss: 5.174 - ETA: 0s - loss: 5.174 - ETA: 0s - loss: 5.165 - ETA: 0s - loss: 5.151 - ETA: 0s - loss: 5.146 - 

KeyboardInterrupt: 

In [ ]:
def weight_opt(oof_nn, oof_rf, y_true):
    weight_nn = np.inf
    best_crps = np.inf
    
    for i in np.arange(0, 1.01, 0.05):
        crps_blend = np.zeros(oof_nn.shape[0])
        for k in range(oof_nn.shape[0]):
            crps_blend[k] = crps_score(i * oof_nn[k,...] + (1-i) * oof_rf[k,...], y_true)
        if np.mean(crps_blend) < best_crps:
            best_crps = np.mean(crps_blend)
            weight_nn = round(i, 2)
            
        print(str(round(i, 2)) + ' : mean crps (Blend) is ', round(np.mean(crps_blend), 6))
        
    print('-'*36)
    print('Best weight for NN: ', weight_nn)
    print('Best weight for LGBM: ', round(1-weight_nn, 2))
#     print('Best weight for RF: ', round(1-weight_nn, 2))
    print('Best mean crps (Blend): ', round(best_crps, 6))
    
    return weight_nn, round(1-weight_nn, 2)

In [ ]:
weight_nn_list =[]
kfold = KFold(5, random_state = 42 + k, shuffle = True)
for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):

    
    tr_x,tr_y = X[tr_inds],y[tr_inds]
    val_x,val_y = X[val_inds],y[val_inds]
    model2,crps = get_model(tr_x,tr_y,val_x,val_y)
    ann_score = model2.predict(val_x)
    ann_score = np.clip(ann_score.cumsum(axis=1), 0, 1)
    
    tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
    vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
    dtrain = lgb.Dataset(tr_x, label= tr_y)
    dvalid = lgb.Dataset(vl_x, label= v_y)
    model1 = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              early_stopping_rounds=10,
                                valid_sets=[dtrain,dvalid],
                              verbose_eval=100,
                              feval=crps_eval)
    lgbm_score = model1.predict(vl_x)
    lgbm_score = np.clip(lgbm_score.cumsum(axis=1), 0, 1)
    
    weight_nn = np.inf
    best_crps = np.inf    
    
    real_y = np.zeros((len(v_y),199))
    for g, v in enumerate(v_y):
        real_y[g, v:] = 1
    
    for i in np.arange(0, 1.01, 0.05):
        crps_blend = np.zeros(val_x.shape[0])
        for k in range(val_x.shape[0]):
            crps_blend[k] = np.mean(( np.clip((i * lgbm_score[k,...] + (1-i) * ann_score[k,...]), 0, 1) - real_y)**2)
        if np.mean(crps_blend) < best_crps:
            best_crps = np.mean(crps_blend)
            weight_nn = round(i, 2)
            
    print(str(round(i, 2)) + ' : mean crps (Blend) is ', round(np.mean(crps_blend), 6))

    print('-'*36)
    print('Best weight for NN: ', weight_nn)
    weight_nn_list.append(weight_nn)
    print('Best weight for LGBM: ', round(1-weight_nn, 2))
    #     print('Best weight for RF: ', round(1-weight_nn, 2))
    print('Best mean crps (Blend): ', round(best_crps, 6))

validation shape 2
Epoch 1/100
18536/18536 [==============================] - ETA: 12s - loss: 5.85 - ETA: 6s - loss: 5.8509 - ETA: 4s - loss: 5.832 - ETA: 3s - loss: 5.815 - ETA: 3s - loss: 5.805 - ETA: 2s - loss: 5.790 - ETA: 2s - loss: 5.777 - ETA: 1s - loss: 5.762 - ETA: 1s - loss: 5.749 - ETA: 1s - loss: 5.735 - ETA: 1s - loss: 5.720 - ETA: 0s - loss: 5.706 - ETA: 0s - loss: 5.691 - ETA: 0s - loss: 5.675 - ETA: 0s - loss: 5.659 - ETA: 0s - loss: 5.645 - ETA: 0s - loss: 5.630 - ETA: 0s - loss: 5.619 - 3s 141us/step - loss: 5.6185

Epoch 00001: CRPS_score_val improved from inf to 0.08303, saving model to best_model.h5
Epoch 2/100
18536/18536 [==============================] - ETA: 1s - loss: 5.325 - ETA: 1s - loss: 5.283 - ETA: 1s - loss: 5.280 - ETA: 1s - loss: 5.263 - ETA: 1s - loss: 5.252 - ETA: 1s - loss: 5.248 - ETA: 1s - loss: 5.227 - ETA: 1s - loss: 5.213 - ETA: 1s - loss: 5.197 - ETA: 0s - loss: 5.193 - ETA: 0s - loss: 5.182 - ETA: 0s - loss: 5.173 - ETA: 0s - loss: 5.157 - 

18536/18536 [==============================] - ETA: 1s - loss: 2.800 - ETA: 1s - loss: 2.783 - ETA: 1s - loss: 2.761 - ETA: 1s - loss: 2.782 - ETA: 1s - loss: 2.781 - ETA: 1s - loss: 2.783 - ETA: 1s - loss: 2.778 - ETA: 1s - loss: 2.777 - ETA: 1s - loss: 2.776 - ETA: 0s - loss: 2.774 - ETA: 0s - loss: 2.771 - ETA: 0s - loss: 2.774 - ETA: 0s - loss: 2.776 - ETA: 0s - loss: 2.772 - ETA: 0s - loss: 2.768 - ETA: 0s - loss: 2.772 - ETA: 0s - loss: 2.774 - ETA: 0s - loss: 2.774 - 2s 117us/step - loss: 2.7749

Epoch 00014: CRPS_score_val improved from 0.01252 to 0.01249, saving model to best_model.h5
Epoch 15/100
18536/18536 [==============================] - ETA: 1s - loss: 2.812 - ETA: 1s - loss: 2.801 - ETA: 1s - loss: 2.773 - ETA: 1s - loss: 2.762 - ETA: 1s - loss: 2.766 - ETA: 1s - loss: 2.776 - ETA: 1s - loss: 2.771 - ETA: 1s - loss: 2.762 - ETA: 1s - loss: 2.759 - ETA: 0s - loss: 2.750 - ETA: 0s - loss: 2.752 - ETA: 0s - loss: 2.755 - ETA: 0s - loss: 2.758 - ETA: 0s - loss: 2.755 - ETA

18536/18536 [==============================] - ETA: 2s - loss: 2.586 - ETA: 1s - loss: 2.628 - ETA: 1s - loss: 2.656 - ETA: 1s - loss: 2.651 - ETA: 1s - loss: 2.653 - ETA: 1s - loss: 2.660 - ETA: 1s - loss: 2.663 - ETA: 1s - loss: 2.665 - ETA: 1s - loss: 2.656 - ETA: 0s - loss: 2.663 - ETA: 0s - loss: 2.665 - ETA: 0s - loss: 2.663 - ETA: 0s - loss: 2.665 - ETA: 0s - loss: 2.667 - ETA: 0s - loss: 2.667 - ETA: 0s - loss: 2.669 - ETA: 0s - loss: 2.671 - ETA: 0s - loss: 2.672 - 2s 117us/step - loss: 2.6715

Epoch 00027: CRPS_score_val did not improve from 0.01230
Epoch 28/100
18536/18536 [==============================] - ETA: 1s - loss: 2.628 - ETA: 1s - loss: 2.672 - ETA: 1s - loss: 2.666 - ETA: 1s - loss: 2.652 - ETA: 1s - loss: 2.647 - ETA: 1s - loss: 2.651 - ETA: 1s - loss: 2.645 - ETA: 1s - loss: 2.645 - ETA: 1s - loss: 2.643 - ETA: 0s - loss: 2.646 - ETA: 0s - loss: 2.642 - ETA: 0s - loss: 2.645 - ETA: 0s - loss: 2.654 - ETA: 0s - loss: 2.656 - ETA: 0s - loss: 2.656 - ETA: 0s - loss

18536/18536 [==============================] - ETA: 2s - loss: 2.569 - ETA: 1s - loss: 2.560 - ETA: 1s - loss: 2.586 - ETA: 1s - loss: 2.578 - ETA: 1s - loss: 2.576 - ETA: 1s - loss: 2.587 - ETA: 1s - loss: 2.589 - ETA: 1s - loss: 2.586 - ETA: 1s - loss: 2.591 - ETA: 0s - loss: 2.596 - ETA: 0s - loss: 2.604 - ETA: 0s - loss: 2.604 - ETA: 0s - loss: 2.605 - ETA: 0s - loss: 2.607 - ETA: 0s - loss: 2.607 - ETA: 0s - loss: 2.607 - ETA: 0s - loss: 2.605 - ETA: 0s - loss: 2.606 - 2s 117us/step - loss: 2.6073

Epoch 00040: CRPS_score_val did not improve from 0.01225
Epoch 41/100
18536/18536 [==============================] - ETA: 2s - loss: 2.574 - ETA: 1s - loss: 2.555 - ETA: 1s - loss: 2.576 - ETA: 1s - loss: 2.564 - ETA: 1s - loss: 2.579 - ETA: 1s - loss: 2.583 - ETA: 1s - loss: 2.584 - ETA: 1s - loss: 2.587 - ETA: 1s - loss: 2.596 - ETA: 0s - loss: 2.602 - ETA: 0s - loss: 2.603 - ETA: 0s - loss: 2.599 - ETA: 0s - loss: 2.595 - ETA: 0s - loss: 2.596 - ETA: 0s - loss: 2.597 - ETA: 0s - loss

18536/18536 [==============================] - ETA: 2s - loss: 2.577 - ETA: 2s - loss: 2.543 - ETA: 1s - loss: 2.523 - ETA: 1s - loss: 2.528 - ETA: 1s - loss: 2.531 - ETA: 1s - loss: 2.531 - ETA: 1s - loss: 2.528 - ETA: 1s - loss: 2.530 - ETA: 1s - loss: 2.530 - ETA: 0s - loss: 2.531 - ETA: 0s - loss: 2.533 - ETA: 0s - loss: 2.536 - ETA: 0s - loss: 2.537 - ETA: 0s - loss: 2.545 - ETA: 0s - loss: 2.540 - ETA: 0s - loss: 2.540 - ETA: 0s - loss: 2.542 - ETA: 0s - loss: 2.543 - 2s 118us/step - loss: 2.5446

Epoch 00054: CRPS_score_val did not improve from 0.01224
Epoch 55/100
18536/18536 [==============================] - ETA: 2s - loss: 2.518 - ETA: 1s - loss: 2.532 - ETA: 1s - loss: 2.545 - ETA: 1s - loss: 2.557 - ETA: 1s - loss: 2.547 - ETA: 1s - loss: 2.550 - ETA: 1s - loss: 2.562 - ETA: 1s - loss: 2.555 - ETA: 1s - loss: 2.554 - ETA: 0s - loss: 2.545 - ETA: 0s - loss: 2.551 - ETA: 0s - loss: 2.554 - ETA: 0s - loss: 2.550 - ETA: 0s - loss: 2.553 - ETA: 0s - loss: 2.549 - ETA: 0s - loss

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 10 rounds
[100]	training's crps: 0.0128141	valid_1's crps: 0.0127408
[200]	training's crps: 0.0116616	valid_1's crps: 0.0124411
[300]	training's crps: 0.0109536	valid_1's crps: 0.0123218
[400]	training's crps: 0.0104986	valid_1's crps: 0.0122661
[500]	training's crps: 0.0101922	valid_1's crps: 0.0122348
[600]	training's crps: 0.00995145	valid_1's crps: 0.0122163
[700]	training's crps: 0.00976035	valid_1's crps: 0.0122016
[800]	training's crps: 0.00962853	valid_1's crps: 0.0121928
[900]	training's crps: 0.00954672	valid_1's crps: 0.0121881
[1000]	training's crps: 0.00949577	valid_1's crps: 0.0121845
Early stopping, best iteration is:
[1033]	training's crps: 0.00948658	valid_1's crps: 0.0121836


In [28]:
real_y = np.zeros((len(v_y),199))
for g, v in enumerate(v_y):
    real_y[g, v:] = 1
for i in np.arange(0, 1.01, 0.05):
        crps_blend = np.zeros(val_x.shape[0])
        for k in range(val_x.shape[0]):
            crps_blend[k] = np.mean(( np.clip((i * lgbm_score[k,...] + (1-i) * ann_score[k,...]), 0, 1) - real_y)**2)
        if np.mean(crps_blend) < best_crps:
            best_crps = np.mean(crps_blend)
            weight_nn = round(i, 2)
            
print(str(round(i, 2)) + ' : mean crps (Blend) is ', round(np.mean(crps_blend), 6))

print('-'*36)
print('Best weight for NN: ', weight_nn)
weight_nn_list.append(weight_nn)
print('Best weight for LGBM: ', round(1-weight_nn, 2))
#     print('Best weight for RF: ', round(1-weight_nn, 2))
print('Best mean crps (Blend): ', round(best_crps, 6))

KeyboardInterrupt: 

In [42]:
( np.clip((i * lgbm_score[k,...] + (1-i) * ann_score[k,...]), 0, 1) - val_y)**2

array([[1.35997665e-12, 6.32606910e-12, 1.62753728e-11, ...,
        9.99929774e-01, 9.99932516e-01, 1.00000000e+00],
       [1.35997665e-12, 6.32606910e-12, 1.62753728e-11, ...,
        9.99929774e-01, 9.99932516e-01, 1.00000000e+00],
       [1.35997665e-12, 6.32606910e-12, 1.62753728e-11, ...,
        9.99929774e-01, 9.99932516e-01, 1.00000000e+00],
       ...,
       [1.35997665e-12, 6.32606910e-12, 1.62753728e-11, ...,
        9.99929774e-01, 9.99932516e-01, 1.00000000e+00],
       [1.35997665e-12, 6.32606910e-12, 1.62753728e-11, ...,
        9.99929774e-01, 9.99932516e-01, 1.00000000e+00],
       [1.35997665e-12, 6.32606910e-12, 1.62753728e-11, ...,
        9.99929774e-01, 9.99932516e-01, 1.00000000e+00]])

In [29]:
real_y

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])